## 잠재 요인 협업 필터링의 개요

* 잠재 요인 협업 필터링은 사용자-아이템 평점 행렬 속에 숨어 있는 잠재 요인을 추출해 추천 예측을 할 수 있게 하는 기법입니다.
* 대규모 다차원 행렬을 SVD와 같은 행렬 분해(Matrix Factorization) 기법으로 분해하는 과정에서 잠재 요인을 추출하는데, 이 잠재요인을 기반으로 사용자-아이템 평점 행렬을 재구성하면서 추천을 구현합니다.

### 잠재 요인 협업 필터링의 이해

* 잠재 요인 협업 필터링은 사용자-아이템 평점 행렬 속에 숨어 있는 잠재 요인을 추출해 추천 예측을 할 수 있게 하는 기법입니다.

* ex) 영화의 잠재 요인 추출
    * 사용자 레벨의 잠재요인
    * 아이템 레벨의 잠재요인

### 행렬 분해를 통한 잠재 요인 협업 필터링

* 잠재 요인 협업 필터링의 행렬 분해
    * $R$ ~$P$ * $Q^T$
        * $R$ : 사용자-아이템 평점 행렬
        * **~** : Decomposed by
        * $P$ : 사용자-잠재 요인 행렬
        * $Q^T$ : 잠재 요인-아이템 행렬
* 잠재 요인 협업 필터링의 행렬 분해 목표는 희소 행렬 형태의 사용자-아이템 평점 행렬을 밀집(Dense)행렬 형태의 사용자-잠재 요인 행렬과 잠재 요인-아이템 행렬로 분해 한 뒤, 이를 재결합하여 밀집 행렬 형태의 사용자-아이템 평점 행렬을 생성하여 사용자에게 새로운 아이템을 추천하는 것입니다.

* 원본 행렬 -> **(행렬 분해)** -> 사용자-잠재요인 행렬 * 잠재 요인-아이템 행렬
* 사용자-잠재요인 행렬 * 잠재요인-아이템 행렬 -> **(내적 곱)** -> 예측 행렬

### 잠재 요인 기반의 행렬 분해 이해

* 원본 행렬(R)
    * 사용자(M) * 아이템(N)
* 사용자-잠재요인 행렬(P)
    * 사용자(M) * 잠재요인(K)
    * 개별 Row는 개별 사용자에 대한 잠재요인(Latent Factor)들을 반형
* 잠재요인-아이템 행렬(Q.T) -> 아이템-잠재요인 행렬의 전치행렬
    * 잠재요인(K) * 아이템(N)
    * 개별 Row는 개별 아이템에 대한 잠재요인(Latent Factor)들을 반형

### 사용자-아이템 평점 행렬 분해 이슈

1. 그러나 SVD는 Missing Value가 없는 행렬에 적용 가능합니다. 따라서 P와 Q 행렬을 일반적인 SVD방식으로는 분해할 수 없습니다.
2. P와 Q를 모르는데 어떻게 R을 예측할 수 있는가?
3. 경사 하강법(Gradient Descent)를 이용하여 P와 Q에 기반한 예측 R값이 실제 R값과 가장 최소의 오류를 가질 수 있도록 비용함수 최적화를 통해 P와 Q를 최적화 유추

### 경사하강법 기반의 행렬 분해


* 경사 하강법을 이용한 행렬 분해 방법은 P와 Q 행렬로 계산된 예측 R 행렬 값이 실제 R 행렬 값과 가장 최소의 오류를 가질 수 있도록 반복적인 비용 함수 최적화를 통해 P와 Q를 유추해내는 것입니다.



* 경사 하강법 기반의 행렬 분해 순서
    1. P와 Q를 임의의 값을 가진 행렬로 설정합니다.
    2. P와 Q.T값을 곱해 예측 R 행렬을 계산하고 예측 R 행렬과 실제 R 행렬에 해당하는 오류 값을 계산합니다.
    3. 이 오류 값을 최소화할 수 있도록, P 와 Q행렬을 적절한 값으로 업데이트합니다.
    4. 만족할 만한 오류 값을 가질 때까지 2,3번 작업을 반복하면서 P와 Q값을 업데이트해 근사화합니다.

### 경사 하강법 기반의 행렬 분해 비용 함수

* 실제 값과 예측값의 오류 최소화와 L2 규제(Regularization)를 고려한 비용 함수식
    * $ min\sum(r_{(u, i)} - p_uq_i^t)^2 + \lambda(\vert\vert q_i \vert\vert^2 + \vert\vert p_u \vert\vert^2) $
    * 실제값과 예측값의 오류 최소화 + 과적합 개선을 위한 L2 규제

### 경사 하강법 기반의 행렬 분해 업데이트 식


* 실제 R 행렬과 예측 R 행렬 값의 차이를 최소화하는 방향성을 가지고 P행렬과 Q행렬에 업데이트 값을 반복적으로 수행하면서 최적화된 예측 R 행렬을 구하는 방식이 경사 하강법 기반의 행렬 분해입니다.


* 비용 함수를 최소화하기 위해서 새롭게 업데이트되는 $p_u'$ 와 $q_i'$
    * $p_u' = p_u + n( e_{(u, i)}*q_i - \lambda*p_u ) $
    * $q_i' = q_i + n( e_{(u, i)}*p_u - \lambda*q_i ) $
        * n -> 학습률
        
        
* $p_u$ : P 행렬의 사용자 u행 벡터
* $q_i^t$ : Q 행렬의 아이템 i행의 전치 벡터(transpose vector)
* $r_{(u, i)}$ : R행렬의 u행, i열에 위치한 값
* $r_{(u, i)}' $ : $ p_u * q_i^t $로 계산하며, u행 i열에 위치한 행렬의 예측값
* $e_{(u, i)}$ : $r_{(u, i)} - r_{(u, i)}' $의 값으로, u행 i열에 위치한 실제 행렬 값과 예측 행렬 값의 차이 오류
* $\alpha$ : SGD 학습률
* $\lambda$ : L2 Regularization 계수